In [1]:

import os, sys, math, time
import numpy as np
import numpy.linalg as la
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd

import json as js
import _pickle as pickle
import bz2
import ray

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from collections import OrderedDict

sys.path.append("../")
import func

In [2]:
torch.cuda.device_count()

1

## Analyse features

In [9]:
data_path = "../../data/"
# load data
data = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")


## DEC-MLP Autoencoder
$
f(x,\theta) = dec(enc(x,\theta_1), \theta_2) = x,   \quad \theta = (\theta_1, \theta_2)
$

$
enc(x, \theta_1) = z, \quad   z \in Z \quad \text{ = latent space}
$

$
dec(z, \theta_2) = x, \quad   x \in X \quad \text{ = input space}
$

This model uses Deep Embedded Clustering (DEC) model for encoder

$
enc = dec(x, \theta) = z, \quad z \in Z
$

$
dec = mlp(X, \theta), \quad \theta = W,b
$

$
mlp(X, W) = f(f(X \cdot w_1 + b_1) \cdot w_2 + b_2) \cdot w_3 + b_3
$

In [7]:
"""
from vlukiyanov on GitHub {https://github.com/vlukiyanov/pt-dec/tree/master/ptdec}
"""
from typing import Iterable, Tuple, Callable, Optional, Union, List
from cytoolz.itertoolz import concat, sliding_window

def build_units(
    dimensions: Iterable[int], activation: Optional[torch.nn.Module]
) -> List[torch.nn.Module]:
    """
    Given a list of dimensions and optional activation, return a list of units where each unit is a linear
    layer followed by an activation layer.
    :param dimensions: iterable of dimensions for the chain
    :param activation: activation layer to use e.g. nn.ReLU, set to None to disable
    :return: list of instances of Sequential
    """

    def single_unit(in_dimension: int, out_dimension: int) -> torch.nn.Module:
        unit = [("linear", nn.Linear(in_dimension, out_dimension))]
        if activation is not None:
            unit.append(("activation", activation))
        return nn.Sequential(OrderedDict(unit))

    return [
        single_unit(embedding_dimension, hidden_dimension)
        for embedding_dimension, hidden_dimension in sliding_window(2, dimensions)
    ]


def default_initialise_weight_bias_(
    weight: torch.Tensor, bias: torch.Tensor, gain: float
) -> None:
    """
    Default function to initialise the weights in a the Linear units of the StackedDenoisingAutoEncoder.
    :param weight: weight Tensor of the Linear unit
    :param bias: bias Tensor of the Linear unit
    :param gain: gain for use in initialiser
    :return: None
    """
    nn.init.xavier_uniform_(weight, gain)
    nn.init.constant_(bias, 0)


class StackedDenoisingAutoEncoder(nn.Module):
    def __init__(
        self,
        dimensions: List[int],
        activation: torch.nn.Module = nn.ReLU(),
        final_activation: Optional[torch.nn.Module] = nn.ReLU(),
        weight_init: Callable[
            [torch.Tensor, torch.Tensor, float], None
        ] = default_initialise_weight_bias_,
        gain: float = nn.init.calculate_gain("relu"),
    ):
        """
        Autoencoder composed of a symmetric decoder and encoder components accessible via the encoder and decoder
        attributes. The dimensions input is the list of dimensions occurring in a single stack
        e.g. [100, 10, 10, 5] will make the embedding_dimension 100 and the hidden dimension 5, with the
        autoencoder shape [100, 10, 10, 5, 10, 10, 100].
        :param dimensions: list of dimensions occurring in a single stack
        :param activation: activation layer to use for all but final activation, default torch.nn.ReLU
        :param final_activation: final activation layer to use, set to None to disable, default torch.nn.ReLU
        :param weight_init: function for initialising weight and bias via mutation, defaults to default_initialise_weight_bias_
        :param gain: gain parameter to pass to weight_init
        """
        super(StackedDenoisingAutoEncoder, self).__init__()
        self.dimensions = dimensions
        self.embedding_dimension = dimensions[0]
        self.hidden_dimension = dimensions[-1]
        # construct the encoder
        encoder_units = build_units(self.dimensions[:-1], activation)
        encoder_units.extend(
            build_units([self.dimensions[-2], self.dimensions[-1]], None)
        )
        self.encoder = nn.Sequential(*encoder_units)
        # construct the decoder
        decoder_units = build_units(reversed(self.dimensions[1:]), activation)
        decoder_units.extend(
            build_units([self.dimensions[1], self.dimensions[0]], final_activation)
        )
        self.decoder = nn.Sequential(*decoder_units)
        # initialise the weights and biases in the layers
        for layer in concat([self.encoder, self.decoder]):
            weight_init(layer[0].weight, layer[0].bias, gain)

    def get_stack(self, index: int) -> Tuple[torch.nn.Module, torch.nn.Module]:
        """
        Given an index which is in [0, len(self.dimensions) - 2] return the corresponding subautoencoder
        for layer-wise pretraining.
        :param index: subautoencoder index
        :return: tuple of encoder and decoder units
        """
        if (index > len(self.dimensions) - 2) or (index < 0):
            raise ValueError(
                "Requested subautoencoder cannot be constructed, index out of range."
            )
        return self.encoder[index].linear, self.decoder[-(index + 1)].linear

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        encoded = self.encoder(batch)
        return self.decoder(encoded)

# Soft Assignment model
class ClusterAssignment(nn.Module):
    def __init__(
        self,
        cluster_number: int,
        embedding_dimension: int,
        alpha: float = 1.0,
        cluster_centers: Optional[torch.Tensor] = None,
    ) -> None:
        """
        Module to handle the soft assignment, for a description see in 3.1.1. in Xie/Girshick/Farhadi,
        where the Student's t-distribution is used measure similarity between feature vector and each
        cluster centroid.
        :param cluster_number: number of clusters
        :param embedding_dimension: embedding dimension of feature vectors
        :param alpha: parameter representing the degrees of freedom in the t-distribution, default 1.0
        :param cluster_centers: clusters centers to initialise, if None then use Xavier uniform
        """
        super(ClusterAssignment, self).__init__()
        self.embedding_dimension = embedding_dimension
        self.cluster_number = cluster_number
        self.alpha = alpha
        if cluster_centers is None:
            initial_cluster_centers = torch.zeros(
                self.cluster_number, self.embedding_dimension, dtype=torch.float
            )
            nn.init.xavier_uniform_(initial_cluster_centers)
        else:
            initial_cluster_centers = cluster_centers
        self.cluster_centers = nn.Parameter(initial_cluster_centers)

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        """
        Compute the soft assignment for a batch of feature vectors, returning a batch of assignments
        for each cluster.
        :param batch: FloatTensor of [batch size, embedding dimension]
        :return: FloatTensor [batch size, number of clusters]
        """
        norm_squared = torch.sum((batch.unsqueeze(1) - self.cluster_centers) ** 2, 2)
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2
        numerator = numerator ** power
        return numerator / torch.sum(numerator, dim=1, keepdim=True)

# DEC Layer
class DEC(nn.Module):
    def __init__(
        self,
        cluster_number: int,
        hidden_dimension: int,
        encoder: torch.nn.Module,
        alpha: float = 1.0,
    ):
        """
        Module which holds all the moving parts of the DEC algorithm, as described in
        Xie/Girshick/Farhadi; this includes the AutoEncoder stage and the ClusterAssignment stage.
        :param cluster_number: number of clusters
        :param hidden_dimension: hidden dimension, output of the encoder
        :param encoder: encoder to use
        :param alpha: parameter representing the degrees of freedom in the t-distribution, default 1.0
        """
        super(DEC, self).__init__()
        self.encoder = encoder
        self.hidden_dimension = hidden_dimension
        self.cluster_number = cluster_number
        self.alpha = alpha
        self.assignment = ClusterAssignment(
            cluster_number, self.hidden_dimension, alpha
        )

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        """
        Compute the cluster assignment using the ClusterAssignment after running the batch
        through the encoder part of the associated AutoEncoder module.
        :param batch: [batch size, embedding dimension] FloatTensor
        :return: [batch size, number of clusters] FloatTensor
        """
        return self.assignment(self.encoder(batch))

    @staticmethod
    def get_p(q:torch.Tensor) -> torch.Tensor:
        p = q**2 / q.sum(0)
        return torch.autograd.Variable((p.t()/p.sum(1)).data, requires_grad=True)

    @staticmethod
    def loss(q:torch.Tensor, p:torch.Tensor):
        return torch.sum(p*torch.log(p/q), dim=-1)


In [8]:
class DEC_AE(nn.Module):
    def __init__(self, encoder:nn.Module, decoder:nn.Module):
        super(DEC_AE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder


    def forward(self, x):
        return self.decoder(self.encoder(x))

In [13]:
# Prepare train data
all_data = []
for d in data:
    d = pickle.loads(d)
    pos = []
    for f in d["frames"]:
        p = [jo["pos"] for jo in f]
        pos.append(p)
    all_data.append(pos)

input_data = np.array([np.concatenate([p for p in j]) for pos in all_data for j in pos])
print(input_data.shape)

(1440, 63)


In [16]:
data_ratio = (.7, .15, .15) # training, validation, testing
SEED = 2021
batch_size = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

x_tensor = torch.from_numpy(input_data).float()
y_tensor = torch.from_numpy(input_data).float()

dataset = TensorDataset(x_tensor, y_tensor)
N = len(dataset)

train_ratio = int(data_ratio[0]*N)
val_ratio = int(data_ratio[1] * N)
test_ratio = int(N-train_ratio-val_ratio)
print("Train: ", train_ratio, ", Validation: ", val_ratio, ", Test: ", test_ratio)

train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(SEED))

train_loader = DataLoader(dataset=train_set, batch_size=batch_size)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size)


cuda
Train:  1007 , Validation:  216 , Test:  217


In [21]:
# Hyper-parameters
input_dim = input_data.shape[1]
output_dim = input_data.shape[1]
latent_dim = 36         # 12 * 3
encoder_layer_sizes = [input_dim, 256, 256, latent_dim]
decoder_layer_sizes = [latent_dim, 256, 256, output_dim]
num_epochs = 100
learning_rate = 0.001
act_fn = "elu"
keep_prob = .2

# model, loss and scheduler
ae = StackedDenoisingAutoEncoder(encoder_layer_sizes, activation=nn.ELU(), final_activation=nn.ELU())
model = DEC_AE(DEC(36, 36, ae.encoder), ae.decoder)

criterion = nn.MSELoss(reduction="mean")
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)

print(model)


DEC_AE(
  (encoder): DEC(
    (encoder): Sequential(
      (0): Sequential(
        (linear): Linear(in_features=63, out_features=256, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (1): Sequential(
        (linear): Linear(in_features=256, out_features=256, bias=True)
        (activation): ELU(alpha=1.0)
      )
      (2): Sequential(
        (linear): Linear(in_features=256, out_features=36, bias=True)
      )
    )
    (assignment): ClusterAssignment()
  )
  (decoder): Sequential(
    (0): Sequential(
      (linear): Linear(in_features=36, out_features=256, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (1): Sequential(
      (linear): Linear(in_features=256, out_features=256, bias=True)
      (activation): ELU(alpha=1.0)
    )
    (2): Sequential(
      (linear): Linear(in_features=256, out_features=63, bias=True)
      (activation): ELU(alpha=1.0)
    )
  )
)


In [22]:
total_step = len(train_loader)
i = 0
n_epochs_no_improve = 5

train_loader_len = float(len(train_loader))
val_loader_len = float(len(val_loader))
test_loader_len = float(len(test_loader))

last_avg_training_loss = 0
min_loss = np.inf
epochs_no_improve = 0
best_model_after_epoch = 0

for epoch in range(num_epochs):
    training_loss = 0
    # training
    for inputs, labels in train_loader:
        # inputs = inputs.to(device)
        # outputs = outputs.to(device)

        pred = model(inputs)
        loss = criterion(pred, labels)
        training_loss+=loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    last_avg_training_loss = training_loss / train_loader_len
    print ('Epoch [{}/{}], Loss: {:.4f}'
        .format(epoch+1, num_epochs, last_avg_training_loss))

    # early stopping
    with torch.no_grad():
        val_loss = 0
        for inputs, labels in val_loader:
            pred_val = model(inputs)
            loss_val = criterion(pred_val, labels)
            val_loss += loss_val.item()

        val_loss /= val_loader_len
        if min_loss > val_loss:
            min_loss = val_loss
            epochs_no_improve = 0
            best_model_after_epoch = epoch

        else:
            epochs_no_improve+=1
            if epochs_no_improve > n_epochs_no_improve:
                print("Early stopping at Epoch: ", epoch)
                print("last training loss: {:2f}".format(last_avg_training_loss))
                print("achieved best validation loss: {:.4f} after at Epoch {}".format(min_loss, best_model_after_epoch))
                break

# Testing
with torch.no_grad():
    test_loss = 0
    for inputs, labels in test_loader:
        pred_test = model(inputs)
        loss_test = criterion(pred_test, labels)
        test_loss += loss_test.item()

    test_loss /= test_loader_len
    print("Test loss: {:.4f}".format(test_loss))


Epoch [1/100], Loss: 0.1884
Epoch [2/100], Loss: 0.1721
Epoch [3/100], Loss: 0.1721
Epoch [4/100], Loss: 0.1721
Epoch [5/100], Loss: 0.1721
Epoch [6/100], Loss: 0.1721
Epoch [7/100], Loss: 0.1721
Early stopping at Epoch:  6
last training loss: 0.172100
achieved best validation loss: 0.1820 after at Epoch 0
Test loss: 0.1849
